In [2]:
import numpy as np
import sys
import matplotlib.pyplot as plt
from import_lib.func import sphere, weierstrass, ackley, rosenbrock, schwefel, griewank, rastrigin
from import_lib.GA import population_init, factorial_cost, factorial_rank, skill_factor_best_task, polynomial_mutation, sbx_crossover
from import_lib.MFEA_Model import MFEA_base, MFEA1
from import_lib.benchmark import CEC17_benchmark_10tasks, GECCO20_benchmark_50tasks

In [3]:
tasks = CEC17_benchmark_10tasks()

In [4]:
print(tasks[0].func(tasks[0].encode(np.array([0] * 50))))
print(tasks[1].func(tasks[1].encode(np.array([80] * 50))))
print(tasks[2].func(tasks[2].encode(np.array([-80] * 50))))
print(tasks[3].func(tasks[3].encode(np.array([-0.4] * 25))))
print(tasks[4].func(tasks[4].encode(np.array([0] * 50))))
print(tasks[5].func(tasks[5].encode(np.array([40] * 50))))
print(tasks[6].func(tasks[6].encode(np.array([-0.4] * 50))))
print(tasks[7].func(tasks[7].encode(np.array([420.9687] * 50))))
print(tasks[8].func(tasks[8].encode(np.append(np.array([-80] * 25), np.array([80] * 25)))))
print(tasks[9].func(tasks[9].encode(np.append(np.array([40] * 25), np.array([-40] * 25)))))


0.0
0.0
0.0
0.0
0.0
4.440892098500626e-16
0.0
0.0006363918728311546
0.0
320000.0


In [5]:
tasks[1].decode(tasks[1].encode(np.array([80] * 50)))

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [6]:
print(tasks[0].encode(np.array([0] * 50))[[0, -1]])
print(tasks[1].encode(np.array([80] * 50))[[0, -1]])
print(tasks[2].encode(np.array([-80] * 50))[[0, -1]])
print(tasks[3].encode(np.array([-0.4] * 25))[[0, -1]])
print(tasks[4].encode(np.array([0] * 50))[[0, -1]])
print(tasks[5].encode(np.array([40] * 50))[[0, -1]])
print(tasks[6].encode(np.array([-0.4] * 50))[[0, -1]])
print(tasks[7].encode(np.array([420.9687] * 50))[[0, -1]])
print(tasks[8].encode(np.append(np.array([-80] * 25), np.array([80] * 25)))[[0, -1]])
print(tasks[9].encode(np.append(np.array([40] * 25), np.array([-40] * 25)))[[0, -1]])

[0.5 0.5]
[0.5 0.5]
[0.5 0.5]
[0.5 0.5]
[0.51 0.51]
[0.5 0.5]
[0.5 0.5]
[0.9209687 0.9209687]
[0.5 0.5]
[ 1.3 -0.3]


In [7]:
fun = sphere(50,shift= 80,limited_space= True, lower_bound= -100, upper_bound= 100),   # 0
fun[0].encode(np.array([180] * 50))[[0, -1]]

array([1., 1.])

In [8]:
test_pop, test_ska = population_init(100, 3, 2)
print(test_pop.shape, test_ska.shape)

(300, 2) (300,)


In [9]:
sbx_crossover(np.array([0, 0, 0.3]), np.array([0.99, 1, 0.7]))

(array([0.        , 0.04729195, 0.31003015]),
 array([1.        , 0.95270805, 0.68996985]))

In [10]:
a = np.array([0, 1, 0.5], dtype= np.float)
print(polynomial_mutation(a, all_dimensions= False))


[0.  1.  0.5]


#### MFEA: SA AND LSA

In [ ]:
def Success_History_Memory_Update(M, next_update_position_M, S, delta, K):

    for i in range(K):
        for j in range(i + 1, K):
            if len(S[i][j]) != 0:
                M[i, j][next_update_position_M[i, j]] =\
                    np.sum(np.array(delta[i][j]) * np.array(S[i][j])**2)/np.sum(np.array(delta[i][j]) * (np.array(S[i][j]))+ 1e-10)
                
                next_update_position_M[i, j] = (next_update_position_M[i, j] + 1) % M.shape[2]

    return M, next_update_position_M

In [ ]:
def LSA_MFEA(MAXEVALS, num_inds_each_task, N_min = 50, tasks = [], nc = 2, nm = 5, H = 30, sigmoid = 0.1, 
            LSA = True, polynomial_all_gen = False):

    #save history of factorial cost
    history_cost = np.empty((0, len(tasks)), np.float)
    
    max_d = 0
    for t in tasks:
        if max_d < t.d:
            max_d = t.d

    N_max = num_inds_each_task
    maxEvals = MAXEVALS/len(tasks)
    eval_k = [0]*len(tasks)
    epoch = 0

    # initial population and skill_factor_arr
    population, skill_factor_arr = population_init(
        N = num_inds_each_task, 
        num_tasks = len(tasks), 
        d = max_d,
        min_val = 0,
        max_val = 1,
    )

    # get skill_factor for pop
    skill_factor_arr = skill_factor_best_task(population, tasks)
    population_factorial_cost = factorial_cost(population, skill_factor_arr, tasks)

    history_cost = np.append(history_cost, [[np.min(population_factorial_cost[np.where(skill_factor_arr == idx)[0]]) for idx in range (len(tasks))]], axis = 0)

    #Inittialize success history mem M
    M = np.ones((len(tasks), len(tasks), H)) / 2
    next_update_position_M = np.zeros((len(tasks), len(tasks)), dtype= np.int)

    # mean and sd of rmp each generations
    # len(task) * len(tasks) * generations * 2
    saved_rmp = np.array([[[[0.5, 0]]]* len(tasks)] * len(tasks)).tolist()

    while np.sum(eval_k) < MAXEVALS:

        S = np.empty((len(tasks), len(tasks), 0)).tolist()
        delta = np.empty((len(tasks), len(tasks), 0)).tolist()

        #initial offspring for epoch
        offspring = np.empty((0, max_d))
        offspring_skill_factor = np.empty((0, ), np.int)
        offspring_factorial_cost = np.empty((0, ), np.int)

        # his rmp of this generation
        rmp_this_gen = np.empty((len(tasks), len(tasks), 0)).tolist()

        while len(offspring_skill_factor) < len(population):
            #random 2 inds
            [idx_pa, idx_pb] = np.random.randint(len(population), size = 2)
            [pa, pb], [skf_pa, skf_pb] = population[[idx_pa, idx_pb]], skill_factor_arr[[idx_pa, idx_pb]]    

            if skf_pa == skf_pb:
                oa, ob = sbx_crossover(pa, pb, nc)

                oa = polynomial_mutation(oa,nm, polynomial_all_gen)
                ob = polynomial_mutation(ob,nm, polynomial_all_gen)
                
                skf_oa, skf_ob = skf_pa, skf_pa

                # Evaluate oa, ob
                fcost_oa, fcost_ob = tasks[skf_oa].func(oa), tasks[skf_ob].func(ob)
                eval_k[skf_oa] += 1
                eval_k[skf_ob] += 1
            else:
                #swap
                if skf_pa > skf_pb: 
                    pa, pb = pb, pa
                    skf_pa, skf_pb = skf_pb, skf_pa

                # caculate rmp
                mu = np.random.choice(M[skf_pa, skf_pb])
                rmp = np.random.normal(mu, sigmoid)
                if rmp < 0: rmp = 0
                if rmp > 1: rmp = 1

                # save generation's rmp
                rmp_this_gen[skf_pa][skf_pb].append(rmp)

                # Inter-task crossover
                if np.random.uniform() < rmp:
                    oa, ob = sbx_crossover(pa, pb, nc)

                    oa = polynomial_mutation(oa,nm, polynomial_all_gen)
                    ob = polynomial_mutation(ob,nm, polynomial_all_gen)
                    
                    skf_oa, skf_ob = np.random.choice([skf_pa, skf_pb], replace=True, size = 2)
                else:
                    # select pa' and pb'
                    idx_pa2 = np.random.choice(np.where(skill_factor_arr == skf_pa)[0])
                    idx_pb2 = np.random.choice(np.where(skill_factor_arr == skf_pb)[0])
                    pa2 = population[idx_pa2]
                    pb2 = population[idx_pb2]

                    oa, _ = sbx_crossover(pa, pa2)
                    ob, _ = sbx_crossover(pb, pb2)

                    oa = polynomial_mutation(oa,nm, polynomial_all_gen)
                    ob = polynomial_mutation(ob,nm, polynomial_all_gen)

                    skf_oa, skf_ob = skf_pa, skf_pb
                
                # Evaluate oa, ob
                fcost_oa, fcost_ob = tasks[skf_oa].func(oa), tasks[skf_ob].func(ob)
                eval_k[skf_oa] += 1
                eval_k[skf_ob] += 1

                # Caculate the maximum imporvement percentage
                Delta = 0
                if (skf_oa == skf_pa):
                    Delta = max(Delta, 
                        (population_factorial_cost[idx_pa] - fcost_oa)/population_factorial_cost[idx_pa]
                    )
                else:
                    Delta = max(Delta, 
                        (population_factorial_cost[idx_pb] - fcost_oa)/population_factorial_cost[idx_pb]
                    )
                if (skf_ob == skf_pa):
                    Delta = max(Delta, 
                        (population_factorial_cost[idx_pa] - fcost_ob)/population_factorial_cost[idx_pa]
                    )
                else:
                    Delta = max(Delta, 
                        (population_factorial_cost[idx_pb] - fcost_ob)/population_factorial_cost[idx_pb]
                    )

                if Delta > 0:
                    S[skf_pa][skf_pb].append(rmp)
                    delta[skf_pa][skf_pb].append(Delta)

            offspring = np.append(offspring, [oa, ob], axis = 0)
            offspring_skill_factor = np.append(offspring_skill_factor, [skf_oa, skf_ob])
            offspring_factorial_cost = np.append(offspring_factorial_cost, [fcost_oa, fcost_ob])
        
        M, next_update_position_M = Success_History_Memory_Update(M, next_update_position_M, S, delta, K = len(tasks))

        # update pop
        population = np.append(population, offspring, axis = 0)
        skill_factor_arr = np.append(skill_factor_arr, offspring_skill_factor, axis = 0)
        population_factorial_cost = np.append(population_factorial_cost, offspring_factorial_cost, axis = 0)

        # Linear_Population_Size_Reduction_Strategy 
        idx_selected_inds = np.empty((0,),dtype=int)
        scalar_fitness = 1/factorial_rank(population_factorial_cost, skill_factor_arr, len(tasks))
        for i in range(len(tasks)): 
            if LSA :
                N_i = int(min(N_max - eval_k[i]/maxEvals * (N_max - N_min), len(np.where(skill_factor_arr == i)[0])))
            else:
                N_i = num_inds_each_task
            ind_task_i = np.where(skill_factor_arr == i)[0]
            sort_idx = ind_task_i[np.argsort(- scalar_fitness[ind_task_i])]
            idx_selected_inds = np.append(idx_selected_inds, sort_idx[:N_i], axis = 0)
        
        population = population[idx_selected_inds]
        skill_factor_arr = skill_factor_arr[idx_selected_inds]
        population_factorial_cost = population_factorial_cost[idx_selected_inds]

        #save history
        history_cost = np.append(history_cost, [[np.min(population_factorial_cost[np.where(skill_factor_arr == idx)[0]]) for idx in range (len(tasks))]], axis = 0)
        
        #save mu and sigmoid rmp
        for i in range (len(tasks)):
            for j in range (i + 1, len(tasks)):
                if len(rmp_this_gen[i][j]) != 0:
                    mu = np.mean(rmp_this_gen[i][j])
                    sd = np.std(rmp_this_gen[i][j])
                    saved_rmp[i][j].append([mu, sd])

        #print
        sys.stdout.write('\r')
        sys.stdout.write('Epoch {}, [%-20s] %3d%% ,pop_size: {}, func_val: {}'
            .format(epoch + 1, len(population),[np.min(population_factorial_cost[np.where(skill_factor_arr == idx)[0]]) for idx in range (len(tasks))])
            % ('=' * ((np.sum(eval_k) + 1) // (MAXEVALS // 20)) + '>' , (np.sum(eval_k) + 1) * 100 // MAXEVALS)
            )
        sys.stdout.flush()

        epoch += 1

    #find solve
    sol_idx = [np.argmin(population_factorial_cost[np.where(skill_factor_arr == idx)]) for idx in range (len(tasks))]
    sol = [task.decode(population[np.where(skill_factor_arr == idx)][sol_idx[idx]]) for idx, task in enumerate(tasks)]

    return sol, history_cost, np.array(saved_rmp)

In [ ]:
mfea_sol, mfea_his = MFEA_base(num_epochs= 1000, num_inds_each_task= 100, tasks = tasks,
                        rmp = 0.3, nc = 2, nm = 5, evaluate_initial_skillFactor= True,
                        one_line=False, num_epochs_printed=10 )

np.save(file = "./RESULTS/mfea_base_his_CEC17.npy", arr = mfea_his )

In [ ]:
sa_sol, sa_his, sa_rmp_his = LSA_MFEA(100000 * len(tasks), LSA = False, num_inds_each_task= 100, tasks= tasks,
                                    nc = 2, nm = 5, H = 30, sigmoid= 0.1)

np.save(file = "./RESULTS/sa_his_CEC17.npy", arr = sa_his)
np.save(file = "./RESULTS/sa_rmp_his_CEC17.npy", arr = sa_rmp_his )

In [ ]:
lsa_sol, lsa_his, lsa_rmp_his = LSA_MFEA(80000 * len(tasks), num_inds_each_task= 100, tasks= tasks, 
                                        nc = 2, nm = 5, H = 30, sigmoid= 0.1)
np.save(file = "./RESULTS/lsa_his_CEC17.npy", arr = lsa_his)
np.save(file = "./RESULTS/lsa_rmp_his_CEC17.npy", arr = lsa_rmp_his )

In [ ]:
mfea_his = np.load(file = "./RESULTS/mfea_base_his_CEC17.npy")

sa_his = np.load(file = "./RESULTS/sa_his_CEC17.npy")
sa_rmp_his  = np.load(file = "./RESULTS/sa_rmp_his_CEC17.npy", allow_pickle= True)

lsa_his = np.load(file = "./RESULTS/lsa_his_CEC17.npy")
lsa_rmp_his  = np.load(file = "./RESULTS/lsa_rmp_his_CEC17.npy", allow_pickle= True)

In [ ]:
fig = plt.figure(figsize = (30, 10), dpi= 200)
fig.suptitle("Compare\n", size = 15)
fig.set_facecolor("white")

label_legend = ["MFEA", "SA", "LSA MFEA"]

for i in range (len(tasks)):
    for idx, sub_his in enumerate([mfea_his, sa_his, lsa_his]): 
        plt.subplot(2, int(len(tasks) / 2), i + 1)

        plt.plot(np.arange(0, 1000, 100), sub_his[np.arange(0, 1000, 100), i], label= label_legend[idx])
        plt.scatter(np.arange(0, 1000, 100), sub_his[np.arange(0, 1000, 100), i])
        plt.legend()

    plt.title(tasks[i].name)
    plt.xlabel("Epoch")
    plt.ylabel("F(x)")
    plt.yscale("log")

In [ ]:
fig = plt.figure(figsize = (30, 20), dpi= 200)
fig.suptitle("SA RMP Val\n", size = 15)
fig.set_facecolor("white")

for i in range (len(tasks)):
    for j in range (len(tasks)):
        r, c = min(i, j), max(i, j)
        if i != j: 
            plt.subplot(int(np.ceil(len(tasks) / 3)), 3, i + 1)
            plt.plot(np.arange(len(sa_rmp_his[r][c])), np.array(sa_rmp_his[r][c])[:, 0], label= 'task: ' +str(j + 1))
            plt.legend()
        else:
            plt.subplot(int(np.ceil(len(tasks) / 3)), 3, i + 1)
            plt.plot(np.arange(1000), np.ones_like(np.arange(1000)), label= 'task: ' +str(j + 1))
            plt.legend()




    plt.title('task ' + str( i + 1))
    plt.xlabel("Epoch")
    plt.ylabel("M_rmp")
    plt.ylim(bottom = -0.1, top = 1.1)


In [ ]:
fig = plt.figure(figsize = (30, 20), dpi= 200)
fig.suptitle("LSA RMP Val\n", size = 15)
fig.set_facecolor("white")

for i in range (len(tasks)):
    for j in range (len(tasks)):
        r, c = min(i, j), max(i, j)
        if i != j: 
            plt.subplot(int(np.ceil(len(tasks) / 3)), 3, i + 1)
            plt.plot(np.arange(1000), np.array(lsa_rmp_his[r][c])[:1000, 0], label= 'task: ' +str(j + 1))
            plt.legend()
        else:
            plt.subplot(int(np.ceil(len(tasks) / 3)), 3, i + 1)
            plt.plot(np.arange(1000), np.ones_like(np.arange(1000)), label= 'task: ' +str(j + 1))
            plt.legend()



    plt.title('task ' + str( i))
    plt.xlabel("Epoch")
    plt.ylabel("M_rmp")
    plt.ylim(bottom = -0.1, top = 1.1)


#### GECCO 20


In [12]:
ID = 1
gecco_tasks = GECCO20_benchmark_50tasks(ID)

In [13]:
mfea_sol, mfea_his = MFEA1(num_epochs= 1000, num_inds_each_task= 100, tasks = gecco_tasks,
                        rmp = 0.3, nc = 2, nm = 5, evaluate_initial_skillFactor= True,
                        one_line=True, num_epochs_printed=10 )

np.save(file = "./RESULTS/mfea_base_his_GECCO20_" + ID + ".npy", arr = mfea_his )

Epoch [100/1000], [==>                 ]  10% ,func_val: [8602.716794930746, 10595.817250781281, 9921.325771357591, 10447.233794062477, 9317.450839271805, 9645.09835816305, 10227.323360693304, 10362.64206885268, 9744.309891823284, 8655.288786260076, 10273.397295757235, 10364.23642279494, 10038.416699365936, 10680.018520536683, 9932.486049102867, 9802.796066436817, 8599.563092247534, 9832.55704745912, 8830.5934687134, 10116.037802341718, 9903.07213902626, 10311.556280302046, 10440.54876008618, 10233.606778396648, 10673.711590948662, 10293.135234712381, 10132.512870584696, 8971.870559969566, 10601.132924961234, 10587.462932108918, 10679.620978772627, 10035.475876013352, 10121.728668708964, 10251.872112429548, 10600.601960553558, 9928.939921156676, 9074.06863461735, 9991.546117684036, 9622.17002851976, 9484.707346168936, 10174.44917013126, 9907.070851474487, 10171.335812148762, 10715.640577219854, 9907.357516684124, 10357.151713508381, 9704.65319362784, 9913.102202708338, 10250.9799958418

In [ ]:
sa_sol, sa_his, sa_rmp_his = LSA_MFEA(100000 * len(gecco_tasks), LSA = False, num_inds_each_task= 100, tasks= gecco_tasks,
                                    nc = 2, nm = 5, H = 30, sigmoid= 0.1)

np.save(file = "./RESULTS/sa_his_GECCO20_" + ID + ".npy", arr = sa_his)
np.save(file = "./RESULTS/sa_rmp_his_GECCO20_" + ID + ".npy", arr = sa_rmp_his )

In [ ]:
lsa_sol, lsa_his, lsa_rmp_his = LSA_MFEA(80000 * len(gecco_tasks), num_inds_each_task= 100, tasks= gecco_tasks, 
                                        nc = 2, nm = 5, H = 30, sigmoid= 0.1)
np.save(file = "./RESULTS/lsa_his_GECCO20_" + ID + ".npy", arr = lsa_his)
np.save(file = "./RESULTS/lsa_rmp_his_GECCO20_" + ID + ".npy", arr = lsa_rmp_his )